In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from pathlib import Path
import json

basepath = Path('/Users/aryamangupta/CricML/Match_Prediction/')
data_folder_path = basepath / 'data' / 'ipl_json'


    
                


        

In [3]:
dataset = []


for json_path in sorted(data_folder_path.glob('*.json')):
    with json_path.open() as f:
        json_data = f.read()
        match_data = json.loads(json_data)
        # print (data['info'])
        
        for inning in match_data['innings']:
            match_dataset = []
            for over in inning['overs']:
                for delivery in over['deliveries']:
                    if 'wickets' in delivery:
                        match_dataset.append('W')
                    else:
                        match_dataset.append(str(delivery['runs']['total']))
            dataset.append(match_dataset)

In [4]:
# print(dataset[0][11])
all_runs = [run for match in dataset for run in match]
runs = sorted(list(set(all_runs)))
print(runs)
stoi = {s:i+1 for i,s in enumerate(runs)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)




[]
{0: '.'}


In [64]:
block_size = 30
X,Y = [], []
for match in dataset:
    #print (match[:11])
    context = [0]* block_size
    for outcome in match:
        ix = stoi[outcome]
        X.append(context)
        Y.append(ix)
        #print(','.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [65]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([260920, 30]), torch.int64, torch.Size([260920]), torch.int64)

In [66]:
C = torch.randn (10,2)
W1 = torch.randn((60,100))
b1 = torch.randn(100)
W2 = torch.randn((100,10))
b2 = torch.randn(10)
parameters = [C, W1, b1, W2, b2]

In [60]:
for p in parameters:
    p.requires_grad = True

In [63]:
for _ in range (10):
    #forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1,60) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1 * p.grad



6.329028129577637
6.051759719848633
5.900296211242676
5.426515579223633
5.228805065155029
5.0076093673706055
4.930269718170166
4.453269004821777
4.243190765380859
4.22750997543335
4.330288410186768
4.416965484619141
3.6963930130004883
3.707919120788574
3.6851186752319336
3.6472973823547363
3.292001962661743
3.2441422939300537
2.954390525817871
2.9557406902313232
2.7031056880950928
2.7652230262756348
2.4557297229766846
2.530885934829712
2.2571029663085938
2.330965280532837
2.088730812072754
2.172010660171509
1.9471226930618286
2.0465028285980225
1.8286080360412598
1.9205775260925293
1.6876355409622192
1.7245845794677734
1.4955064058303833
1.4669914245605469
1.3349974155426025
1.290486454963684
1.2130588293075562
1.1599292755126953
1.1272655725479126
1.0473865270614624
1.0523293018341064
0.9372158050537109
0.9396460652351379
0.8284508585929871
0.829930305480957
0.7334187626838684
0.7319384217262268
0.6494004130363464
0.6399528980255127
0.5741868019104004
0.5587348341941833
0.513780653476